Deployment is on : https://github.com/hdans/DataNiaga

# Import Important Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
from collections import Counter
from scipy import stats
plt.style.use('seaborn-v0_8')
import math

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor

from mlxtend.frequent_patterns import fpgrowth, association_rules

# Import Dataset

In [ ]:
df = pd.read_excel("data_retail2.xlsx")

# Data Understanding

## Change Invoice Date to DateTime

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

start_date = pd.to_datetime("2022-01-02")
target_date = pd.to_datetime("2025-12-14")

time_delta = target_date - start_date
df['InvoiceDate'] = df['InvoiceDate'] + time_delta

## Dataset Info

In [ ]:
df.info()

## Dataset Describe

In [ ]:
df.describe()

## Dataset Null Values

In [ ]:
df.isnull().sum()

## Dataset Unique Values

In [ ]:
df['InvoiceNo'].nunique()

In [ ]:
obj_col = df.select_dtypes(include='object').columns
for col in obj_col:
    print(f"{col}: {df[col].nunique()} unique values")
    print(df[col].value_counts())

In [ ]:
df['BRANCH_SPLR'].nunique()

In [ ]:
df['BRANCH_SPLR'].unique()

In [ ]:
df['warehouseProductsID'].nunique()

In [ ]:
df['warehouseProductsID'].unique()

In [ ]:
df['CHANNELID_SPLR'].nunique()

In [ ]:
df['CHANNELNAME_SPLR'].unique()

In [ ]:
df[df['CHANNELNAME_SPLR'] == 'Salon Kecantikan']

In [ ]:
df['CHANNELNAME_SPLR'].value_counts()

In [ ]:
df['CustomerID'].nunique()

In [ ]:
df['oldCUSTID'].nunique()

In [ ]:
df['PRODUCT'].nunique()

In [ ]:
df['PRODUCT_CATEGORY'].nunique()

In [ ]:
df['PRODUCT_CATEGORY'].value_counts()

In [ ]:
df['InvoiceDate'] = df['InvoiceDate'].dt.date

In [ ]:
df['PROVINSI'].unique()

In [ ]:
df[df['PROVINSI'].isna()]['KOTA'].value_counts(dropna=False)

# Data Preprocessing

## Normalize City Column

In [ ]:
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SOLO 08960358885', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'NGUNTORONADI', 'PROVINSI'] = 'JAWA TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'JATIROTO', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'MERBAUNG-KLATEN SELA', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'BATURETNO', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'PRMBNAN KLT', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'NGAWEN KLATEN', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'BULIKERTO WONOGIRI', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'KARTASURA', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'NGADIROJO WONOGIRI', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'TIRTOMOYO WONOGIRI', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'MOJOSONGO SURAKARTA', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'KLATEN UTARA', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'POLOKERTO', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'KARTOSURO', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SURAKARTA', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SUKOHARJO', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'KARANGANYAR', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'BOYOLALI', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SOLO', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SANGATA', 'PROVINSI'] = 'KALIMANTAN TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SRAGEN', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'BONDOWOSO', 'PROVINSI'] = 'JAWA TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'WONOGIRI', 'PROVINSI'] = 'JAWA TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SITUBONDO', 'PROVINSI'] = 'JAWA TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'JEMBER', 'PROVINSI'] = 'JAWA TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'BANYUWANGI', 'PROVINSI'] = 'JAWA TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'SAMPIT', 'PROVINSI'] = 'KALIMANTAN TENGAH'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'BONTANG', 'PROVINSI'] = 'KALIMANTAN TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'LUMAJANG', 'PROVINSI'] = 'JAWA TIMUR'
df.loc[df['KOTA'].astype(str).str.strip().str.upper() == 'KLATEN', 'PROVINSI'] = 'JAWA TENGAH'

In [ ]:
df[df['KOTA'].isna() & df['PROVINSI'].isna()]

## Take Not Null Values of City & Province

In [ ]:
df = df[df['KOTA'].notna() | df['PROVINSI'].notna()]

## Casefold the Province

In [ ]:
df['PROVINSI'] = df['PROVINSI'].str.strip().str.upper()

In [ ]:
df['PROVINSI'].unique()

## Normalize Province Column

In [ ]:
df['PROVINSI'] = df['PROVINSI'].replace('KAL-BAR', 'KALIMANTAN BARAT')
df['PROVINSI'] = df['PROVINSI'].replace('KAL-SEL', 'KALIMANTAN SELATAN')
df['PROVINSI'] = df['PROVINSI'].replace('KALTENG', 'KALIMANTAN TENGAH')
df['PROVINSI'] = df['PROVINSI'].replace('JAWATENGAH', 'JAWA TENGAH')
df['PROVINSI'] = df['PROVINSI'].replace('DIY', 'DI YOGYAKARTA')

In [ ]:
df['PROVINSI'].unique()

## Make Island Column from Province Information

In [ ]:
map_pulau = {
    'KALIMANTAN TIMUR': 'KALIMANTAN & SULAWESI',
    'KALIMANTAN BARAT': 'KALIMANTAN & SULAWESI',
    'KALIMANTAN SELATAN': 'KALIMANTAN & SULAWESI',
    'KALIMANTAN TENGAH': 'KALIMANTAN & SULAWESI',
    'JAWA TENGAH': 'JAWA, BALI, & NT',
    'JAWA TIMUR': 'JAWA, BALI, & NT',
    'JAWA BARAT': 'JAWA, BALI, & NT',
    'BANTEN' : 'JAWA, BALI, & NT',
    'DI YOGYAKARTA': 'JAWA, BALI, & NT',
    'BALI' : 'JAWA, BALI, & NT',
    'NUSA TENGGARA BARAT': 'JAWA, BALI, & NT',
    'BANGKA BELITUNG': 'SUMATERA & RIAU',
    'SUMATERA SELATAN': 'SUMATERA & RIAU',
    'SUMATERA UTARA': 'SUMATERA & RIAU',
    'SUMATERA BARAT': 'SUMATERA & RIAU',
    'RIAU': 'SUMATERA & RIAU',
    'LAMPUNG': 'SUMATERA & RIAU',
    'JAMBI': 'SUMATERA & RIAU',
    'KEPULAUAN RIAU': 'SUMATERA & RIAU',
    'LAMPUNG': 'SUMATERA & RIAU',
    'KEPULAUAN RIAU': 'SUMATERA & RIAU',
    'RIAU': 'SUMATERA & RIAU',
    'SULAWESI SELATAN': 'KALIMANTAN & SULAWESI',
    'SULAWESI UTARA': 'KALIMANTAN & SULAWESI',
    'SULAWESI TENGGARA': 'KALIMANTAN & SULAWESI',
    'SULAWESI TENGAH': 'KALIMANTAN & SULAWESI',
}

In [ ]:
df['PULAU'] = df['PROVINSI'].map(map_pulau)

In [ ]:
start = df['InvoiceDate'].min()
end = df['InvoiceDate'].max()
all_dates = set([start + timedelta(days=x) for x in range((end-start).days + 1)])

In [ ]:
df.columns

## Fix the Minus Quanitity Sold

In [ ]:
df['Quantity'] = df['Quantity'].apply(lambda x : -x if x < 0 else x)

# Exploratory Data Analysis (EDA)

## Plot Category By General (Indonesia)

In [ ]:
categories = sorted(df['PRODUCT_CATEGORY'].unique())
all_dates = pd.date_range(start=pd.to_datetime(start), end=pd.to_datetime(end))

for product_cat in categories:
    tmp = df[df['PRODUCT_CATEGORY'] == product_cat].copy()

    if tmp.empty:
        continue

    tmp['InvoiceDate'] = pd.to_datetime(tmp['InvoiceDate'])

    # A. DATA HARIAN
    daily = tmp.groupby(tmp['InvoiceDate'].dt.date)['Quantity'].sum().reset_index()
    daily.columns = ['InvoiceDate', 'qty_sum']
    daily.index = pd.to_datetime(daily['InvoiceDate'])
    daily_full = daily.reindex(all_dates, method=None)

    # C. DATA MINGGUAN
    weekly = tmp.groupby(pd.Grouper(key='InvoiceDate', freq='W'))['Quantity'].sum()
    # B. DATA BULANAN
    monthly = tmp.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['Quantity'].sum()


    # Plot
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))

    # Daily
    axes[0].plot(daily_full.index, daily_full['qty_sum'], color='tab:blue', linestyle='-', linewidth=1)
    axes[0].set_title(f'Daily Quantity: {product_cat}', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Quantity')
    axes[0].grid(alpha=0.3)

    # Weekly
    axes[2].plot(weekly.index, weekly.values, color='tab:green', marker='s', linestyle='-', linewidth=2)
    axes[2].set_title(f'Weekly Quantity: {product_cat}', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Date')
    axes[2].set_ylabel('Quantity')
    axes[2].grid(alpha=0.3)
    
    # Monthly
    axes[1].plot(monthly.index, monthly.values, color='tab:red', marker='o', linestyle='-', linewidth=2)
    axes[1].set_title(f'Monthly Quantity: {product_cat}', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Quantity')
    axes[1].grid(alpha=0.3)


    plt.tight_layout()
    plt.show()


If plotted by General (Indonesia), the movement plot are not significantly cause a pattern

## Plot Category By Province

In [ ]:
# df = df.dropna(subset=['PROVINSI'])
# df['PROVINSI'] = df['PROVINSI'].astype(str)
# df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# if 'start' not in locals(): start = df['InvoiceDate'].min()
# if 'end' not in locals(): end = df['InvoiceDate'].max()

# all_dates = pd.date_range(start=start, end=end)

# list_provinsi = sorted(df['PROVINSI'].unique())

# for provinsi in list_provinsi:
#     df_prov = df[df['PROVINSI'] == provinsi].copy()
#     categories = sorted(df_prov['PRODUCT_CATEGORY'].unique())
    
#     for product_cat in categories:
#         tmp = df_prov[df_prov['PRODUCT_CATEGORY'] == product_cat].copy()
#         if tmp.empty:
#             continue

#         daily = tmp.groupby(tmp['InvoiceDate'].dt.normalize())['Quantity'].sum()
#         daily_full = daily.reindex(all_dates, fill_value=0)

#         weekly = tmp.groupby(pd.Grouper(key='InvoiceDate', freq='W'))['Quantity'].sum()
#         monthly = tmp.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['Quantity'].sum()

#         fig, axes = plt.subplots(1, 3, figsize=(20, 5))
#         plt.suptitle(f"PROVINSI: {provinsi} | KATEGORI: {product_cat}", fontsize=16, fontweight='bold', y=1.05)

#         axes[0].plot(daily_full.index, daily_full.values, linewidth=1, alpha=0.8)
#         axes[0].set_title('Daily Quantity', fontsize=12, fontweight='bold')
#         axes[0].set_xlabel('Date')
#         axes[0].set_ylabel('Qty')
#         axes[0].grid(alpha=0.3)
#         axes[0].tick_params(axis='x', rotation=45)

#         axes[1].plot(weekly.index, weekly.values, marker='s', linestyle='-', linewidth=2)
#         axes[1].set_title('Weekly Quantity', fontsize=12, fontweight='bold')
#         axes[1].set_xlabel('Date')
#         axes[1].grid(alpha=0.3)
#         axes[1].tick_params(axis='x', rotation=45)

#         axes[2].plot(monthly.index, monthly.values, marker='o', linestyle='-', linewidth=2)
#         axes[2].set_title('Monthly Quantity', fontsize=12, fontweight='bold')
#         axes[2].set_xlabel('Date')
#         axes[2].grid(alpha=0.3)
#         axes[2].tick_params(axis='x', rotation=45)

#         plt.tight_layout()
#         plt.show()

If we grouping by Province, it's showed that there are so many days/weeks that has zero sales, it's too noice.. But, if we use Monthly Sales by Province, the data is too short

## Plot Category By Pulau

In [ ]:
df = df.dropna(subset=['PULAU'])
df['PULAU'] = df['PULAU'].astype(str)

if 'start' not in locals(): start = df['InvoiceDate'].min()
if 'end' not in locals(): end = df['InvoiceDate'].max()

all_dates = pd.date_range(start=pd.to_datetime(start), end=pd.to_datetime(end))

list_pulau = sorted(df['PULAU'].unique())

for pulau in list_pulau:
    df_pulau = df[df['PULAU'] == pulau].copy()
    categories = sorted(df_pulau['PRODUCT_CATEGORY'].unique())
    
    for product_cat in categories:
        tmp = df_pulau[df_pulau['PRODUCT_CATEGORY'] == product_cat].copy()
        if tmp.empty:
            continue
    
        daily = tmp.groupby(tmp['InvoiceDate'].dt.date)['Quantity'].sum()
        daily.index = pd.to_datetime(daily.index)
    
        daily_full = daily.reindex(all_dates, fill_value=0) 
        weekly = tmp.groupby(pd.Grouper(key='InvoiceDate', freq='W'))['Quantity'].sum()
        monthly = tmp.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['Quantity'].sum()

        fig, axes = plt.subplots(1, 3, figsize=(20, 5))
        plt.suptitle(f"PULAU: {pulau} | KATEGORI: {product_cat}", fontsize=16, fontweight='bold', y=1.05)
    
        axes[0].plot(daily_full.index, daily_full.values, color='tab:blue', linewidth=1, alpha=0.8)
        axes[0].set_title('Daily Quantity', fontsize=12, fontweight='bold')
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('Qty')
        axes[0].grid(alpha=0.3)
        axes[0].tick_params(axis='x', rotation=45)
    
        axes[1].plot(weekly.index, weekly.values, color='tab:green', marker='s', linestyle='-', linewidth=2)
        axes[1].set_title('Weekly Quantity', fontsize=12, fontweight='bold')
        axes[1].set_xlabel('Date')
        axes[1].grid(alpha=0.3)
        axes[1].tick_params(axis='x', rotation=45)

        axes[2].plot(monthly.index, monthly.values, color='tab:red', marker='o', linestyle='-', linewidth=2)
        axes[2].set_title('Monthly Quantity', fontsize=12, fontweight='bold')
        axes[2].set_xlabel('Date')
        axes[2].grid(alpha=0.3)
        axes[2].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()

Grouping it by Pulau in Weekly Sales is the best solution, cause it's serve a long trends, and serve a good trends too

## Payday Impact on Trends

In [ ]:
sns.set_palette("husl")

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

def is_payday_week(date):
    day = date.day
    return 1 if (day >= 25 or day <= 5) else 0

df['Is_Payday_Week'] = df['InvoiceDate'].apply(is_payday_week)
df['Week'] = df['InvoiceDate'].dt.isocalendar().week
df['Month'] = df['InvoiceDate'].dt.month

weekly_agg = df.groupby([pd.Grouper(key='InvoiceDate', freq='W'), 'PULAU']).agg({
    'Quantity': 'sum',
    'Is_Payday_Week': 'first'
}).reset_index()

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('ANALISIS VOLATILITAS PERMINTAAN PER WILAYAH PULAU', fontsize=16, fontweight='bold', y=0.995)

ax1 = axes[0, 0]
pulau_order = weekly_agg.groupby('PULAU')['Quantity'].median().sort_values(ascending=False).index
sns.boxplot(data=weekly_agg, x='PULAU', y='Quantity', order=pulau_order, ax=ax1, palette='Set2')
ax1.set_title('Distribusi Volume Transaksi Mingguan per Pulau', fontweight='bold', fontsize=12)
ax1.set_xlabel('Pulau')
ax1.set_ylabel('Quantity (Unit)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

ax2 = axes[0, 1]
volatility = weekly_agg.groupby('PULAU')['Quantity'].agg(['mean', 'std'])
volatility['CV'] = (volatility['std'] / volatility['mean']) * 100
volatility = volatility.sort_values('CV', ascending=False)

colors = ['red' if cv > 50 else 'orange' if cv > 30 else 'green' for cv in volatility['CV']]
ax2.barh(volatility.index, volatility['CV'], color=colors, edgecolor='black')
ax2.set_title('Coefficient of Variation (Volatilitas Permintaan)', fontweight='bold', fontsize=12)
ax2.set_xlabel('CV (%)')
ax2.set_ylabel('Pulau')
ax2.axvline(x=50, color='red', linestyle='--', linewidth=1, label='High Volatility (>50%)')
ax2.axvline(x=30, color='orange', linestyle='--', linewidth=1, label='Medium Volatility (30-50%)')
ax2.legend(fontsize=9)
ax2.grid(axis='x', alpha=0.3)

ax3 = axes[1, 0]
payday_comparison = weekly_agg.groupby(['PULAU', 'Is_Payday_Week'])['Quantity'].mean().unstack(fill_value=0)
payday_comparison['Lift'] = ((payday_comparison[1] - payday_comparison[0]) / payday_comparison[0]) * 100
payday_comparison = payday_comparison.sort_values('Lift', ascending=False)

x = np.arange(len(payday_comparison))
width = 0.35
ax3.bar(x - width/2, payday_comparison[0], width, label='Non-Payday Week', color='skyblue', edgecolor='black')
ax3.bar(x + width/2, payday_comparison[1], width, label='Payday Week', color='salmon', edgecolor='black')
ax3.set_title('Efek Payday: Perbandingan Volume Transaksi', fontweight='bold', fontsize=12)
ax3.set_xlabel('Pulau')
ax3.set_ylabel('Avg Quantity (Unit)')
ax3.set_xticks(x)
ax3.set_xticklabels(payday_comparison.index, rotation=45, ha='right')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

ax4 = axes[1, 1]
colors_lift = ['darkgreen' if lift > 20 else 'yellowgreen' if lift > 10 else 'gray' for lift in payday_comparison['Lift']]
ax4.barh(payday_comparison.index, payday_comparison['Lift'], color=colors_lift, edgecolor='black')
ax4.set_title('Payday Lift Effect (%)', fontweight='bold', fontsize=12)
ax4.set_xlabel('% Increase During Payday')
ax4.set_ylabel('Pulau')
ax4.axvline(x=20, color='darkgreen', linestyle='--', linewidth=1, label='Strong Effect (>20%)')
ax4.axvline(x=10, color='yellowgreen', linestyle='--', linewidth=1, label='Moderate Effect (10-20%)')
ax4.legend(fontsize=9)
ax4.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## Seasonality

In [ ]:
fig2, axes2 = plt.subplots(3, 1, figsize=(18, 14))
fig2.suptitle('KARAKTERISTIK MUSIMAN: POLA TREN MINGGUAN PER PULAU', fontsize=16, fontweight='bold')

top_3_pulau = weekly_agg.groupby('PULAU')['Quantity'].sum().nlargest(3).index

for idx, pulau in enumerate(top_3_pulau):
    ax = axes2[idx]
    
    subset = weekly_agg[weekly_agg['PULAU'] == pulau].copy()
    subset = subset.sort_values('InvoiceDate')
    
    ax.plot(subset['InvoiceDate'], subset['Quantity'], marker='o', linewidth=2, markersize=4, label='Quantity')
    
    payday_weeks = subset[subset['Is_Payday_Week'] == 1]
    ax.scatter(payday_weeks['InvoiceDate'], payday_weeks['Quantity'], 
               color='red', s=100, zorder=5, label='Payday Week', marker='^', edgecolors='black')
    
    subset['Rolling_Avg'] = subset['Quantity'].rolling(window=4, min_periods=1).mean()
    ax.plot(subset['InvoiceDate'], subset['Rolling_Avg'], linestyle='--', linewidth=2, 
            color='orange', label='4-Week Moving Avg', alpha=0.7)
    
    ax.set_title(f'Pulau: {pulau}', fontweight='bold', fontsize=12)
    ax.set_xlabel('Date')
    ax.set_ylabel('Quantity (Unit)')
    ax.legend(loc='upper left')
    ax.grid(alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Heatmap Correlation

In [ ]:
fig3, ax = plt.subplots(figsize=(16, 8))

heatmap_data = weekly_agg.pivot_table(index='PULAU', columns=weekly_agg['InvoiceDate'].dt.strftime('%Y-%W'), 
                                       values='Quantity', aggfunc='sum', fill_value=0)

heatmap_normalized = heatmap_data.apply(lambda x: (x - x.mean()) / x.std(), axis=1)

sns.heatmap(heatmap_normalized, cmap='RdYlGn', center=0, annot=False, fmt='.1f', 
            linewidths=0.5, cbar_kws={'label': 'Z-Score (Deviation from Mean)'}, ax=ax)
ax.set_title('HEATMAP: Peak Demand per Minggu & Pulau (Z-Score Normalized)', fontweight='bold', fontsize=14)
ax.set_xlabel('Week (Year-Week)')
ax.set_ylabel('Pulau')
plt.xticks(rotation=90, fontsize=8)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print(" RINGKASAN STATISTIK VOLATILITAS PER PULAU")
print("="*70)
summary = volatility[['mean', 'std', 'CV']].copy()
summary.columns = ['Avg Weekly Vol', 'Std Dev', 'Volatility (CV%)']
summary['Payday Lift (%)'] = payday_comparison['Lift']
print(summary.sort_values('Volatility (CV%)', ascending=False).to_string(float_format='%.2f'))
print("="*70)

## Product Frequent Analysis

In [ ]:
print("=" * 80)
print(" ANALISIS FREKUENSI KATEGORI PRODUK - ANCHOR PRODUCTS")
print("=" * 80)

category_freq_global = df['PRODUCT_CATEGORY'].value_counts()
print("\n📊 TOP 15 KATEGORI PRODUK (Global - Semua Pulau):")
print(category_freq_global.head(15))

penetrasi_pulau = df.groupby('PRODUCT_CATEGORY')['PULAU'].nunique().reset_index()
penetrasi_pulau.columns = ['PRODUCT_CATEGORY', 'Num_Pulau']
penetrasi_pulau['Penetrasi_%'] = (penetrasi_pulau['Num_Pulau'] / df['PULAU'].nunique()) * 100

anchor_products = category_freq_global.reset_index()
anchor_products.columns = ['PRODUCT_CATEGORY', 'Frequency']
anchor_products = anchor_products.merge(penetrasi_pulau, on='PRODUCT_CATEGORY')
anchor_products = anchor_products.sort_values(['Penetrasi_%', 'Frequency'], ascending=[False, False])

print("\n🎯 ANCHOR PRODUCTS (Penetrasi Pasar Tinggi di Semua Pulau):")
anchor_threshold = anchor_products[anchor_products['Penetrasi_%'] >= 66.67]  
print(anchor_threshold[['PRODUCT_CATEGORY', 'Frequency', 'Num_Pulau', 'Penetrasi_%']].head(20))

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

top_20_freq = anchor_products.head(20)
axes[0].barh(top_20_freq['PRODUCT_CATEGORY'], top_20_freq['Frequency'], color='steelblue', edgecolor='black')
axes[0].set_xlabel('Jumlah Transaksi', fontsize=11, fontweight='bold')
axes[0].set_title('Top 20 Kategori Produk (Frekuensi)', fontsize=12, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(axis='x', alpha=0.3)

colors_penetrasi = ['darkgreen' if p == 100 else 'orange' if p >= 66.67 else 'lightcoral' 
                    for p in top_20_freq['Penetrasi_%']]
axes[1].barh(top_20_freq['PRODUCT_CATEGORY'], top_20_freq['Penetrasi_%'], color=colors_penetrasi, edgecolor='black')
axes[1].set_xlabel('Penetrasi Pasar (%)', fontsize=11, fontweight='bold')
axes[1].set_title('Top 20 Kategori Produk (Penetrasi Pasar)', fontsize=12, fontweight='bold')
axes[1].axvline(x=100, color='darkgreen', linestyle='--', linewidth=2, label='100% (Semua Pulau)')
axes[1].axvline(x=66.67, color='orange', linestyle='--', linewidth=2, label='67% (Anchor Threshold)')
axes[1].invert_yaxis()
axes[1].legend()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## Buying Pattern Analysis

In [ ]:
print("\n" + "=" * 80)
print("DETEKSI POLA PEMBELIAN SERENTAK (Co-occurrence Analysis)")
print("=" * 80)

co_purchase_pairs = []
invoice_products = df.groupby('InvoiceNo')['PRODUCT_CATEGORY'].apply(list).reset_index()

print(f"\nTotal Invoice: {len(invoice_products)}")
print(f"Invoice dengan Multiple Items (Bundling): {(invoice_products['PRODUCT_CATEGORY'].apply(len) > 1).sum()}")

for items in invoice_products['PRODUCT_CATEGORY']:
    if len(items) >= 2:
        unique_items = list(set(items)) 
        
        if len(unique_items) >= 2:
            for i in range(len(unique_items)):
                for j in range(i + 1, len(unique_items)):
                    pair = tuple(sorted([unique_items[i], unique_items[j]]))
                    co_purchase_pairs.append(pair)

pair_counter = Counter(co_purchase_pairs)
top_pairs = pair_counter.most_common(20)

print("\nTOP 20 PASANGAN PRODUK (Co-occurrence dalam 1 Invoice):")
print("-" * 80)
for idx, (pair, count) in enumerate(top_pairs, 1):
    print(f"{idx:2d}. {pair[0]:25s} + {pair[1]:25s} : {count:5d} transaksi")

co_purchase_df = pd.DataFrame([
    {'Product_1': pair[0], 'Product_2': pair[1], 'Co_Purchase_Count': count}
    for pair, count in top_pairs
])

fig, ax = plt.subplots(figsize=(14, 8))
pair_labels = [f"{p[0][:15]}...\n+ {p[1][:15]}..." for p, c in top_pairs]
pair_counts = [c for p, c in top_pairs]
colors_pairs = plt.cm.viridis(np.linspace(0, 1, len(top_pairs)))

ax.barh(range(len(top_pairs)), pair_counts, color=colors_pairs, edgecolor='black')
ax.set_yticks(range(len(top_pairs)))
ax.set_yticklabels(pair_labels, fontsize=9)
ax.set_xlabel('Frekuensi Co-Purchase', fontsize=11, fontweight='bold')
ax.set_title('Top 20 Pasangan Produk yang Dibeli Bersamaan', fontsize=13, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## Pruning Data

In [ ]:
print("\n" + "=" * 80)
print(" 3. VALIDASI PRUNING DATA - EFISIENSI MEMORI")
print("=" * 80)

category_transaction_count = df['PRODUCT_CATEGORY'].value_counts()

PRUNING_THRESHOLD = 5

prune_candidates = category_transaction_count[category_transaction_count < PRUNING_THRESHOLD]
keep_products = category_transaction_count[category_transaction_count >= PRUNING_THRESHOLD]

print(f"\nSTATISTIK PRUNING:")
print(f"   Total Kategori Unik: {len(category_transaction_count)}")
print(f"   Kategori dengan Frekuensi >= {PRUNING_THRESHOLD}: {len(keep_products)}")
print(f"   Kategori dengan Frekuensi < {PRUNING_THRESHOLD} (PRUNED): {len(prune_candidates)}")

print(f"\nPRODUK DENGAN FREKUENSI SANGAT RENDAH (< {PRUNING_THRESHOLD} transaksi):")
print("-" * 80)
if not prune_candidates.empty:
    for product, freq in prune_candidates.items():
        print(f"   {product:35s} : {freq:3d} transaksi")
else:
    print("   (Tidak ada produk dengan frekuensi < 5)")

total_rows = len(df)
rows_to_prune = df[df['PRODUCT_CATEGORY'].isin(prune_candidates.index)].shape[0]
pruning_ratio = (rows_to_prune / total_rows) * 100

print(f"\nESTIMASI EFISIENSI MEMORI:")
print(f"   Total Baris Dataset: {total_rows:,}")
print(f"   Baris yang Akan Di-Prune: {rows_to_prune:,} ({pruning_ratio:.2f}%)")
print(f"   Baris Setelah Pruning: {total_rows - rows_to_prune:,}")

print(f"\nVALIDASI SUPPORT THRESHOLD untuk Market Basket Analysis:")
print(f"   Min Support (1% default): {len(df) * 0.01:.0f} transaksi")
print(f"   Min Support (0.1%): {len(df) * 0.001:.0f} transaksi")
print(f"   Produk dengan < {PRUNING_THRESHOLD} transaksi TIDAK akan lolos support 1%")
print(f"   ➜ Pruning ini AMAN untuk dilakukan di backend FastAPI")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].hist(category_transaction_count.values, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(x=PRUNING_THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Pruning Threshold ({PRUNING_THRESHOLD})')
axes[0].set_xlabel('Frekuensi Transaksi', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Jumlah Kategori Produk', fontsize=11, fontweight='bold')
axes[0].set_title('Distribusi Frekuensi Kategori Produk (Semua)', fontsize=12, fontweight='bold')
axes[0].set_yscale('log')
axes[0].legend()
axes[0].grid(alpha=0.3)

before_after = pd.DataFrame({
    'Status': ['Keep\n(Freq >= 5)', 'Prune\n(Freq < 5)'],
    'Row_Count': [total_rows - rows_to_prune, rows_to_prune],
    'Product_Count': [len(keep_products), len(prune_candidates)]
})

ax2 = axes[1]
colors_pie = ['#2ecc71', '#e74c3c']
wedges, texts, autotexts = ax2.pie(before_after['Row_Count'], labels=before_after['Status'], 
                                     autopct='%1.1f%%', colors=colors_pie, startangle=90, 
                                     explode=(0.05, 0.05), textprops={'fontsize': 11, 'fontweight': 'bold'})
ax2.set_title(f'Efek Pruning pada Total Baris Dataset', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Modeling

In [ ]:
df_pulau = df.copy()

In [ ]:
df_apriori = df_pulau.copy()

In [ ]:
df_pulau = df.drop(['BRANCH_SPLR', 'BRANCHNAME_SPLR',
    'warehouseProductsID', 'BARCODEID', 'StockCode', 'PRODUCT', 'UnitPrice', 'oldCUSTID',
    'CustomerID', 'CUSTNAME', 'ADDRESS', 'KOTA', 'PROVINSI', 'NEGARA',
    'CHANNELID_SPLR', 'CHANNELNAME_SPLR', 'SUBDISTID', 'SUBDIST_NAME', 'UnitPriceRupiah'], axis=1)

In [ ]:
df = df.drop(['InvoiceNo', 'BRANCH_SPLR', 'BRANCHNAME_SPLR',
    'warehouseProductsID', 'BARCODEID', 'StockCode', 'PRODUCT', 'UnitPrice', 'oldCUSTID',
    'CustomerID', 'CUSTNAME', 'ADDRESS', 'KOTA', 'PROVINSI', 'NEGARA',
    'CHANNELID_SPLR', 'CHANNELNAME_SPLR', 'SUBDISTID', 'SUBDIST_NAME', 'UnitPriceRupiah', 'PULAU'], axis=1)

In [ ]:
df_pulau['InvoiceDate'] = pd.to_datetime(df_pulau['InvoiceDate'])

## Directive LGBM (global predict)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df_grouped = df.groupby([pd.Grouper(key='InvoiceDate', freq='W'), 'PRODUCT_CATEGORY'])['Quantity'].sum()
df_wide = df_grouped.unstack(level='PRODUCT_CATEGORY').resample('W').asfreq()
df_interpolated = df_wide.interpolate(method='linear')
df = df_interpolated.stack().reset_index()
df.columns = ['InvoiceDate', 'PRODUCT_CATEGORY', 'Quantity']
df_pivot = df.pivot(index='InvoiceDate', columns='PRODUCT_CATEGORY', values='Quantity')

TEST_WEEKS = 12
LOOK_BACK = 4 

X_all = []
y_all = []
categories_list = []
dates_list = []

for category in df_pivot.columns:
    series = df_pivot[category].values
    series_log = np.log1p(series)
    
    for i in range(LOOK_BACK, len(series_log) - TEST_WEEKS + 1):
        past_window = series_log[i-LOOK_BACK : i]
        future_window = series_log[i : i+TEST_WEEKS] 
        
        if len(future_window) < TEST_WEEKS:
            continue
            
        feat_mean = np.mean(past_window)
        feat_std = np.std(past_window)
    
        features = list(past_window) + [feat_mean, feat_std]
        
        X_all.append(features)
        y_all.append(future_window)
        categories_list.append(category)
        dates_list.append(df_pivot.index[i])

X_df = pd.DataFrame(X_all)
feat_cols = [f'Lag_{j}' for j in range(LOOK_BACK, 0, -1)] + ['Mean', 'Std']
X_df.columns = feat_cols
X_df['Category'] = categories_list
X_df['Category'] = X_df['Category'].astype('category')
X_df['Month'] = pd.to_datetime(dates_list).month

y_arr = np.array(y_all)

cutoff_date = df_pivot.index[-TEST_WEEKS]
is_train = pd.to_datetime(dates_list) < cutoff_date
is_test = pd.to_datetime(dates_list) == cutoff_date 

X_train = X_df[is_train]
y_train = y_arr[is_train]
X_test = X_df[is_test]
y_test = y_arr[is_test] 

lgbm = LGBMRegressor(n_estimators=500, learning_rate=0.07, num_leaves=20, n_jobs=-1)
model = MultiOutputRegressor(lgbm)
model.fit(X_train, y_train)

y_pred_log = model.predict(X_test)

y_pred_final = np.expm1(y_pred_log)
y_test_final = np.expm1(y_test)

test_categories = X_test['Category'].values
eval_results = []

plt.figure(figsize=(18, 10))

for i, cat_name in enumerate(test_categories):
    actual = y_test_final[i]
    pred = y_pred_final[i]
    
    pred = np.maximum(pred, 0)
    
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mape = mean_absolute_percentage_error(actual, pred)
    mae = mean_absolute_error(actual, pred) 
    
    note = ""
    if mape > 0.5 and mae < 5:
        note = "(Low Vol)"
    
    eval_results.append({
        'Category': cat_name,
        'RMSE': rmse,
        'MAE': mae,          
        'MAPE': mape,
        'Note': note
    })
    
    if i < 12:
        plt.subplot(3, 4, i+1)
        weeks = range(1, TEST_WEEKS+1)
        plt.plot(weeks, actual, label='Actual', marker='o', color='green')
        plt.plot(weeks, pred, label='Pred', marker='x', linestyle='--', color='red')
        plt.title(f"{cat_name}\nMAPE: {mape:.1%} | MAE: {mae:.1f}")
        plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

df_eval = pd.DataFrame(eval_results)
df_eval['MAPE_Percentage'] = df_eval['MAPE'].apply(lambda x: f"{x:.2%}")
print("\n=== FINAL RESULTS (LOG TRANSFORMED) ===")
print(df_eval.sort_values(by='MAPE')[['Category', 'MAE', 'MAPE_Percentage', 'Note']])

## Local Predict

In [ ]:
df_pulau['InvoiceDate'] = pd.to_datetime(df_pulau['InvoiceDate'])
df_grouped = df_pulau.groupby([
    pd.Grouper(key='InvoiceDate', freq='W'), 
    'PULAU', 
    'PRODUCT_CATEGORY'
])['Quantity'].sum().reset_index()

df_pivot = df_grouped.pivot(index='InvoiceDate', columns=['PULAU', 'PRODUCT_CATEGORY'], values='Quantity')

df_pivot = df_pivot.resample('W').asfreq().interpolate(method='linear').fillna(0)

df_stacked = df_pivot.stack(level=['PULAU', 'PRODUCT_CATEGORY']).reset_index()
df_stacked.columns = ['InvoiceDate', 'PULAU', 'PRODUCT_CATEGORY', 'Quantity']

TEST_WEEKS = 10
LOOK_BACK = 4

def is_payday_week(date):
    day = date.day
    if day >= 25 or day <= 5:
        return 1
    return 0

X_all = []
y_all = []
meta_info = [] 
dates_list = []

unique_combinations = df_stacked[['PULAU', 'PRODUCT_CATEGORY']].drop_duplicates().values

print("Sedang memproses Feature Engineering...")

for pulau, category in unique_combinations:
    subset = df_stacked[
        (df_stacked['PULAU'] == pulau) & 
        (df_stacked['PRODUCT_CATEGORY'] == category)
    ].sort_values('InvoiceDate')
    
    series_raw = subset['Quantity'].values
    
    for i in range(LOOK_BACK, len(series_raw) - TEST_WEEKS + 1):
        past_window = series_raw[i-LOOK_BACK : i]
        future_window = series_raw[i : i+TEST_WEEKS]
        
        if len(future_window) < TEST_WEEKS:
            continue
            
        feat_mean = np.mean(past_window)
        feat_std = np.std(past_window)
        
        target_date = subset['InvoiceDate'].iloc[i]
        feat_payday = is_payday_week(target_date)
        
        features = list(past_window) + [feat_mean, feat_std, feat_payday]
        
        X_all.append(features)
        y_all.append(future_window)
        
        meta_info.append({'PULAU': pulau, 'CATEGORY': category})
        dates_list.append(target_date)

X_df = pd.DataFrame(X_all)
feat_cols = [f'Lag_{j}' for j in range(LOOK_BACK, 0, -1)] + ['Mean', 'Std', 'Is_Payday']
X_df.columns = feat_cols

meta_df = pd.DataFrame(meta_info)
X_df['PULAU'] = meta_df['PULAU'].astype('category')
X_df['CATEGORY'] = meta_df['CATEGORY'].astype('category')
X_df['Month'] = pd.to_datetime(dates_list).month

y_arr = np.array(y_all)

cutoff_date = df_stacked['InvoiceDate'].max() - pd.Timedelta(weeks=TEST_WEEKS) + pd.Timedelta(days=1)
dates_series = pd.to_datetime(dates_list)

is_train = dates_series < cutoff_date
is_test = dates_series == dates_series.max()

X_train = X_df[is_train]
y_train = y_arr[is_train]
X_test = X_df[is_test]
y_test = y_arr[is_test]

print(f"Data Train: {len(X_train)} samples")
print(f"Data Test : {len(X_test)} samples")

print("Melatih Model dengan Objective Tweedie...")

lgbm = LGBMRegressor(
    objective='tweedie',            
    tweedie_variance_power=1.5,  
    n_estimators=1000, 
    learning_rate=0.05, 
    num_leaves=31, 
    n_jobs=-1
)

model = MultiOutputRegressor(lgbm)
model.fit(X_train, y_train)
print("Training Selesai.")

y_pred_final = model.predict(X_test)
y_test_final = y_test

eval_results = []

test_info = meta_df[is_test].reset_index(drop=True)

for i in range(len(test_info)):
    actual = y_test_final[i]
    pred = y_pred_final[i]
    
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    
    if np.sum(actual) == 0:
        mape = 0.0
    else:
        mape = mean_absolute_percentage_error(actual, pred)
        
    eval_results.append({
        'PULAU': test_info.iloc[i]['PULAU'],
        'CATEGORY': test_info.iloc[i]['CATEGORY'],
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape
    })

unique_pulau = test_info['PULAU'].unique()

print(f"\nMembuat visualisasi untuk {len(unique_pulau)} Wilayah Pulau...")

for p_name in unique_pulau:
    indices = test_info[test_info['PULAU'] == p_name].index
    
    num_plots = len(indices)
    cols = 10                         
    rows = math.ceil(num_plots / cols)
    
    fig, axes = plt.subplots(
        rows, 
        cols, 
        figsize=(cols * 3, rows * 3),  
        dpi=300                        
    )
    
    axes = axes.flatten()
    
    fig.suptitle(
        f"WILAYAH: {p_name}\n(Actual vs Tweedie Prediction - 10 Weeks)",
        fontsize=18,
        fontweight='bold',
        y=1.02
    )
    
    for i, idx in enumerate(indices):
        cat_name = test_info.iloc[idx]['CATEGORY']
        
        actual = y_test_final[idx]
        pred = y_pred_final[idx]
        
        metric = next(
            (item for item in eval_results 
             if item['PULAU'] == p_name and item['CATEGORY'] == cat_name),
            None
        )
        
        mape_val = metric['MAPE']
        mae_val = metric['MAE']
        
        ax = axes[i]
        weeks = range(1, TEST_WEEKS + 1)
        
        ax.plot(
            weeks, actual,
            label='Actual',
            marker='o',
            linewidth=1.5,
            markersize=3
        )
        ax.plot(
            weeks, pred,
            label='Pred',
            linestyle='--',
            marker='x',
            linewidth=1.5,
            markersize=3
        )
        
        title_color = 'red' if mape_val > 0.5 else 'black'
        ax.set_title(
            f"{cat_name}\nMAPE:{mape_val:.0%} | MAE:{mae_val:.1f}",
            fontsize=8,
            fontweight='bold',
            color=title_color
        )
        
        ax.grid(alpha=0.2)
        ax.tick_params(axis='both', labelsize=7)
        ax.set_xlabel('Week', fontsize=7)
        
        if i == 0:
            ax.legend(fontsize=7, loc='upper left')

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()
    
    print(f"   Selesai plotting {p_name}.")

In [ ]:
df_eval = pd.DataFrame(eval_results)
mean_sales_map = {}

if 'test_info' in locals() and 'y_test_final' in locals():
    for i in range(len(test_info)):
        p_name = test_info.iloc[i]['PULAU']
        c_name = test_info.iloc[i]['CATEGORY']
        mean_val = np.mean(y_test_final[i]) 
        mean_sales_map[(p_name, c_name)] = mean_val
else:
    print("Warning: Variabel 'test_info' atau 'y_test_final' hilang. Pastikan cell training sudah dijalankan.")

df_eval['Actual_Mean'] = df_eval.apply(lambda x: mean_sales_map.get((x['PULAU'], x['CATEGORY']), 0), axis=1)

def get_quality_score(row):
    mae = row['MAE']
    mean_sales = row['Actual_Mean']
    
    if mean_sales < 10:
        if mae <= 1.0: return "⭐⭐⭐⭐ Excellent"  
        elif mae <= 2.0: return "⭐⭐⭐ Good"       
        elif mae <= 3.5: return "⭐⭐ Fair"       
        else: return "⚠️ Poor"                  
        
    else:
        if mean_sales == 0: return "⚠️ Poor"
        
        ratio = mae / mean_sales
        if ratio <= 0.15: return "⭐⭐⭐⭐ Excellent"     
        elif ratio <= 0.25: return "⭐⭐⭐ Good"        
        elif ratio <= 0.40: return "⭐⭐ Fair"          
        else: return "⚠️ Poor"                          

df_eval['Quality'] = df_eval.apply(get_quality_score, axis=1)

cols_show = ['PULAU', 'CATEGORY', 'Actual_Mean', 'MAE', 'Quality']

print("="*80)
print(" LAPORAN KUALITAS MODEL FORECASTING (PER PULAU & KATEGORI)")
print("="*80)

print("\n✅ TOP PERFORMERS (Model Sangat Akurat)")
print("-" * 80)
good_performance = df_eval[df_eval['Quality'].str.contains("Excellent|Good")]
if not good_performance.empty:
    print(good_performance[cols_show].sort_values(by=['PULAU', 'Quality'], ascending=[True, True]).to_string(index=False, formatters={
        'Actual_Mean': '{:.1f}'.format,
        'MAE': '{:.1f}'.format
    }))
else:
    print("Tidak ada kategori Excellent/Good.")

print("\n\nNEEDS ATTENTION (Model Kurang Akurat / Data Volatile)")
print("-" * 80)
bad_performance = df_eval[df_eval['Quality'].str.contains("Fair|Poor")]
if not bad_performance.empty:
    print(bad_performance[cols_show].sort_values(by=['PULAU', 'Quality'], ascending=[True, False]).to_string(index=False, formatters={
        'Actual_Mean': '{:.1f}'.format,
        'MAE': '{:.1f}'.format
    }))
else:
    print("Tidak ada kategori Fair/Poor.")

print("\n" + "="*80)
print(" RINGKASAN PERFORMA")
print("-" * 80)
print(df_eval['Quality'].value_counts())

## Global Predict 10 Weeks Forward

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df_grouped = df.groupby([pd.Grouper(key='InvoiceDate', freq='W'), 'PRODUCT_CATEGORY'])['Quantity'].sum()
df_wide = df_grouped.unstack(level='PRODUCT_CATEGORY').resample('W').asfreq()
df_interpolated = df_wide.interpolate(method='linear')
df = df_interpolated.stack().reset_index()
df.columns = ['InvoiceDate', 'PRODUCT_CATEGORY', 'Quantity']
df_pivot = df.pivot(index='InvoiceDate', columns='PRODUCT_CATEGORY', values='Quantity')

FORECAST_WEEKS = 10 
LOOK_BACK = 4

X_all = []
y_all = []
categories_list = []

for category in df_pivot.columns:
    series = df_pivot[category].values
    series_log = np.log1p(series)
    
    for i in range(LOOK_BACK, len(series_log) - FORECAST_WEEKS + 1):
        past_window = series_log[i-LOOK_BACK : i]
        future_window = series_log[i : i+FORECAST_WEEKS]
        
        if len(future_window) < FORECAST_WEEKS:
            continue
            
        feat_mean = np.mean(past_window)
        feat_std = np.std(past_window)
        features = list(past_window) + [feat_mean, feat_std]
        
        X_all.append(features)
        y_all.append(future_window)
        categories_list.append(category)

X_train_full = pd.DataFrame(X_all)
feat_cols = [f'Lag_{j}' for j in range(LOOK_BACK, 0, -1)] + ['Mean', 'Std']
X_train_full.columns = feat_cols
X_train_full['Category'] = categories_list
X_train_full['Category'] = X_train_full['Category'].astype('category')
y_train_full = np.array(y_all)

lgbm = LGBMRegressor(n_estimators=1000, learning_rate=0.07, num_leaves=20, n_jobs=-1)
model = MultiOutputRegressor(lgbm)
model.fit(X_train_full, y_train_full)

last_date = df_pivot.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(weeks=1), periods=FORECAST_WEEKS, freq='W')
future_forecasts = {}

for category in df_pivot.columns:
    latest_series_log = np.log1p(df_pivot[category].values[-LOOK_BACK:])
    feat_mean = np.mean(latest_series_log)
    feat_std = np.std(latest_series_log)
    features = list(latest_series_log) + [feat_mean, feat_std]
    
    input_row = pd.DataFrame([features], columns=feat_cols)
    input_row['Category'] = category
    input_row['Category'] = input_row['Category'].astype('category')
    
    pred_log = model.predict(input_row)[0]
    pred_final = np.expm1(pred_log)
    future_forecasts[category] = np.maximum(pred_final, 0)

df_history_long = df_pivot.reset_index().melt(id_vars='InvoiceDate', var_name='PRODUCT_CATEGORY', value_name='Quantity')
df_history_long['Data_Type'] = 'Actual'

df_future_wide = pd.DataFrame(future_forecasts, index=future_dates)
df_future_long = df_future_wide.reset_index().melt(id_vars='index', var_name='PRODUCT_CATEGORY', value_name='Quantity')
df_future_long.rename(columns={'index': 'InvoiceDate'}, inplace=True)
df_future_long['Data_Type'] = 'Forecast'

df_combined = pd.concat([df_history_long, df_future_long], axis=0).sort_values(by=['PRODUCT_CATEGORY', 'InvoiceDate']).reset_index(drop=True)

print("=== CONTOH DATA GABUNGAN (Head & Tail) ===")
print(df_combined.head(3))
print("...")
print(df_combined.tail(3))
print("-" * 50)

categories_to_plot = df_combined['PRODUCT_CATEGORY'].unique()
n_cols = 4
n_rows = int(np.ceil(len(categories_to_plot) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
axes = axes.flatten()

for i, category in enumerate(categories_to_plot):
    ax = axes[i]
    subset = df_combined[df_combined['PRODUCT_CATEGORY'] == category]
    
    actual_data = subset[subset['Data_Type'] == 'Actual']
    forecast_data = subset[subset['Data_Type'] == 'Forecast']
    
    last_actual = actual_data.iloc[-1]
    connector = pd.DataFrame([last_actual])
    connector['Data_Type'] = 'Forecast' 
    forecast_plot_data = pd.concat([connector, forecast_data])
    
    plot_start_date = actual_data['InvoiceDate'].max() - pd.Timedelta(weeks=24)
    actual_plot = actual_data[actual_data['InvoiceDate'] >= plot_start_date]
    
    ax.plot(actual_plot['InvoiceDate'], actual_plot['Quantity'], label='Actual', color='tab:blue', marker='o', markersize=4)
    
    ax.plot(forecast_plot_data['InvoiceDate'], forecast_plot_data['Quantity'], label='Forecast', color='tab:red', linestyle='--', marker='x', markersize=4, linewidth=2)
    
    ax.set_title(f"{category}", fontsize=10, fontweight='bold')
    ax.grid(alpha=0.3)
    ax.tick_params(axis='x', rotation=45, labelsize=8)
    
    if i == 0: ax.legend()

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## Local Predict 10 Weeks Forward

In [ ]:
df_pulau['InvoiceDate'] = pd.to_datetime(df_pulau['InvoiceDate'])
df_grouped = df_pulau.groupby([
    pd.Grouper(key='InvoiceDate', freq='W'), 
    'PULAU', 
    'PRODUCT_CATEGORY'
])['Quantity'].sum().reset_index()

df_pivot = df_grouped.pivot(index='InvoiceDate', columns=['PULAU', 'PRODUCT_CATEGORY'], values='Quantity')
df_pivot = df_pivot.resample('W').asfreq().interpolate(method='linear').fillna(0)
df_stacked = df_pivot.stack(level=['PULAU', 'PRODUCT_CATEGORY']).reset_index()
df_stacked.columns = ['InvoiceDate', 'PULAU', 'PRODUCT_CATEGORY', 'Quantity']

FORECAST_WEEKS = 10  
LOOK_BACK = 4        

def is_payday_week(date):
    day = date.day
    if day >= 25 or day <= 5: return 1
    return 0

unique_pulau = df_stacked['PULAU'].unique()
future_dates = pd.date_range(start=df_stacked['InvoiceDate'].max() + pd.Timedelta(weeks=1), periods=FORECAST_WEEKS, freq='W')

history_data_list = []
forecast_data_list = []

print(f"Memulai Training Full Data & Forecasting {FORECAST_WEEKS} Minggu ke Depan...")

for p_name in unique_pulau:
    print(f"-> Memproses Pulau: {p_name}")
    
    df_subset = df_stacked[df_stacked['PULAU'] == p_name].copy()
    
    df_subset['Data_Type'] = 'Actual'
    history_data_list.append(df_subset)
    
    X_train_all = []
    y_train_all = []
    meta_train = []
    
    cats_in_pulau = df_subset['PRODUCT_CATEGORY'].unique()
    for category in cats_in_pulau:
        sub_cat = df_subset[df_subset['PRODUCT_CATEGORY'] == category].sort_values('InvoiceDate')
        series = sub_cat['Quantity'].values
        
        series_log = np.log1p(series)
        
        for i in range(LOOK_BACK, len(series_log) - FORECAST_WEEKS + 1):
            past_window = series_log[i-LOOK_BACK : i]
            future_window = series_log[i : i+FORECAST_WEEKS]
            
            if len(future_window) < FORECAST_WEEKS: continue
            
            feat_mean = np.mean(past_window)
            feat_std = np.std(past_window)
            
            target_date = sub_cat['InvoiceDate'].iloc[i]
            feat_payday = is_payday_week(target_date)
            
            features = list(past_window) + [feat_mean, feat_std, feat_payday]
            
            X_train_all.append(features)
            y_train_all.append(future_window)
            meta_train.append({'CATEGORY': category})

    X_df = pd.DataFrame(X_train_all)
    feat_cols = [f'Lag_{j}' for j in range(LOOK_BACK, 0, -1)] + ['Mean', 'Std', 'Is_Payday']
    X_df.columns = feat_cols
    
    meta_df = pd.DataFrame(meta_train)
    X_df['CATEGORY'] = meta_df['CATEGORY'].astype('category')
    
    y_arr = np.array(y_train_all)
    
    lgbm = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=20, n_jobs=-1)
    model = MultiOutputRegressor(lgbm)
    model.fit(X_df, y_arr)
    
    for category in cats_in_pulau:
        sub_cat = df_subset[df_subset['PRODUCT_CATEGORY'] == category].sort_values('InvoiceDate')
        last_series = sub_cat['Quantity'].values[-LOOK_BACK:]
        last_series_log = np.log1p(last_series)
        
        feat_mean = np.mean(last_series_log)
        feat_std = np.std(last_series_log)
        
        feat_payday = is_payday_week(future_dates[0])
        
        features = list(last_series_log) + [feat_mean, feat_std, feat_payday]
        
        input_row = pd.DataFrame([features], columns=feat_cols)
        input_row['CATEGORY'] = category
        input_row['CATEGORY'] = input_row['CATEGORY'].astype('category')
        
        pred_log = model.predict(input_row)[0]
        pred_final = np.expm1(pred_log)
        pred_final = np.maximum(pred_final, 0) 
        
        for d, val in zip(future_dates, pred_final):
            forecast_data_list.append({
                'InvoiceDate': d,
                'PULAU': p_name,
                'PRODUCT_CATEGORY': category,
                'Quantity': val,
                'Data_Type': 'Forecast'
            })

print("Proses Selesai.")

df_history = pd.concat(history_data_list)
df_forecast = pd.DataFrame(forecast_data_list)

df_combined_pulau = pd.concat([df_history, df_forecast], axis=0).sort_values(['PULAU', 'PRODUCT_CATEGORY', 'InvoiceDate'])
df_combined_pulau = df_combined_pulau.reset_index(drop=True)

print(f"\nTotal Baris Data Gabungan: {len(df_combined_pulau)}")
print("Contoh Data:")
print(df_combined_pulau.tail())

for p_name in unique_pulau:
    subset_plot = df_combined_pulau[df_combined_pulau['PULAU'] == p_name]
    categories = subset_plot['PRODUCT_CATEGORY'].unique()
    
    num_plots = len(categories)
    cols = 10
    rows = math.ceil(num_plots / cols)
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
    axes = axes.flatten()
    
    fig.suptitle(f"WILAYAH: {p_name}\n(History + Future Forecast 10 Weeks)", fontsize=20, fontweight='bold', y=1.005)
    
    for i, cat in enumerate(categories):
        ax = axes[i]
        
        data_cat = subset_plot[subset_plot['PRODUCT_CATEGORY'] == cat]
        
        actual_data = data_cat[data_cat['Data_Type'] == 'Actual']
        forecast_data = data_cat[data_cat['Data_Type'] == 'Forecast']
        
        if not actual_data.empty and not forecast_data.empty:
            last_act = actual_data.iloc[-1]
            connector = pd.DataFrame([last_act]).copy()
            connector['Data_Type'] = 'Forecast_Connector' 
            forecast_plot = pd.concat([connector, forecast_data])
        else:
            forecast_plot = forecast_data
        
        start_plot = actual_data['InvoiceDate'].max() - pd.Timedelta(weeks=24)
        actual_plot = actual_data[actual_data['InvoiceDate'] >= start_plot]
        
        ax.plot(actual_plot['InvoiceDate'], actual_plot['Quantity'], label='Actual', color='tab:blue', marker='o', markersize=4)
        ax.plot(forecast_plot['InvoiceDate'], forecast_plot['Quantity'], label='Forecast', color='tab:red', linestyle='--', marker='x', markersize=4)
        
        ax.set_title(f"{cat}", fontsize=10, fontweight='bold')
        ax.grid(alpha=0.3)
        ax.tick_params(axis='x', rotation=45, labelsize=8)
        
        if i == 0: ax.legend(loc='upper left')
        
    for j in range(i+1, len(axes)): fig.delaxes(axes[j])
    plt.tight_layout()
    plt.show()

In [ ]:
df_combined_pulau

## FP-Growth Algorithm

In [ ]:
print("Membersihkan Data...")
df_apriori['PRODUCT_CATEGORY'] = df_apriori['PRODUCT_CATEGORY'].astype(str).str.strip()

df_apriori.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df_apriori['InvoiceNo'] = df_apriori['InvoiceNo'].astype(str)
df_apriori = df_apriori[df_apriori['Quantity'] > 0]

print(f"Total Transaksi Bersih: {len(df_apriori)}")
print("-" * 50)

def analyze_basket_per_pulau(dataframe, pulau_name, min_sup=0.01, min_lift=1.0):
    print(f"\n🛒 Memproses Market Basket: {pulau_name}...")
    
    df_subset = dataframe[dataframe['PULAU'] == pulau_name].copy()
    
    if df_subset.empty:
        print("   -> Data kosong.")
        return None

    item_counts = df_subset['PRODUCT_CATEGORY'].value_counts()
    valid_items = item_counts[item_counts >= 5].index
    df_subset = df_subset[df_subset['PRODUCT_CATEGORY'].isin(valid_items)]

    basket = df_subset.pivot_table(index='InvoiceNo', columns='PRODUCT_CATEGORY', values='Quantity', aggfunc='sum').fillna(0)

    basket_sets = basket.applymap(lambda x: True if x > 0 else False)
    
    if 'POSTAGE' in basket_sets.columns:
        basket_sets.drop('POSTAGE', inplace=True, axis=1)

    print(f"   -> Dimensi Matrix: {basket_sets.shape}")

    try:
        frequent_itemsets = fpgrowth(basket_sets, min_support=min_sup, use_colnames=True)
    except MemoryError:
        print("   ❌ Masih Memory Error. Coba naikkan min_support.")
        return None
    
    if frequent_itemsets.empty:
        print("   -> Tidak ada itemset yang memenuhi support.")
        return None

    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift)
    
    if rules.empty:
        print("   -> Tidak ada rules yang memenuhi lift.")
        return None
        
    rules['PULAU'] = pulau_name
    rules = rules.sort_values(['lift', 'confidence'], ascending=[False, False])
    
    return rules

unique_pulau = df_apriori['PULAU'].unique()
all_rules_list = []

MIN_SUPPORT = 0.1
MIN_LIFT = 2.0

for pulau in unique_pulau:
    rules_df = analyze_basket_per_pulau(df_apriori, pulau, min_sup=MIN_SUPPORT, min_lift=MIN_LIFT)
    
    if rules_df is not None:
        all_rules_list.append(rules_df)
        print("   -> Top 3 Rules:")
        print(rules_df[['antecedents', 'consequents', 'lift']].head(3).to_string(index=False))

if all_rules_list:
    final_mba_results = pd.concat(all_rules_list, ignore_index=True)
    
    final_mba_results['antecedents'] = final_mba_results['antecedents'].apply(lambda x: ', '.join(list(x)))
    final_mba_results['consequents'] = final_mba_results['consequents'].apply(lambda x: ', '.join(list(x)))
    
    print("\n✅ ANALISIS SELESAI!")
    print(f"Total Rules: {len(final_mba_results)}")
    
    print("\n=== TOP 10 RULES TERKUAT (GLOBAL) ===")
    print(final_mba_results[['PULAU', 'antecedents', 'consequents', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head(10))
else:
    print("\n❌ Tidak ada rules ditemukan. Coba turunkan min_support sedikit.")

In [ ]:
final_mba_results

In [ ]:
final_mba_results['PULAU'].value_counts()

In [ ]:
print(f"Total Rules Sebelum Filter: {len(final_mba_results)}")

In [ ]:
rules_viz = final_mba_results[
    (final_mba_results['support'] >= 0.10) &
    (final_mba_results['lift']    >= 2.0)
].copy()

rules_viz = rules_viz.sort_values(by='support', ascending=False).head(20)

rules_viz['Rule_Label'] = rules_viz['antecedents'] + " → " + rules_viz['consequents']

plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    rules_viz['confidence'],
    rules_viz['support'],
    c=rules_viz['lift'],
    s=rules_viz['lift'] * 80,   
    cmap='plasma',
    alpha=0.8,
    edgecolor='k'
)

for _, row in rules_viz.head(10).iterrows():
    plt.text(
        row['confidence'],
        row['support'],
        row['Rule_Label'],
        fontsize=8,
        ha='left',
        va='bottom'
    )

plt.colorbar(scatter, label='Lift')
plt.title('Top Frequent Itemsets (min_support ≥ 0.10, lift > 2.0)', fontsize=14, fontweight='bold')
plt.xlabel('Confidence')
plt.ylabel('Support')
plt.grid(alpha=0.3)

plt.show()